# Historical distribution: Forecast

### Setup

In [ ]:
import glob

import numpy as np
import pandas as pd
from scipy.stats import nbinom

from config import FORECAST_DATES, ROOT
from src.realtime_utils import load_realtime_training_data

In [2]:
# Function to fit a negative binomial distribution
def fit_negative_binomial(values):
    mean = np.mean(values)
    var = np.var(values)
    if var > mean:  # Ensure valid parameters
        p = mean / var
        r = mean**2 / (var - mean)
        return r, p
    else:
        return None, None  # Return None if not valid

In [3]:
def forecast_historical_average(df, forecast_date, window=2, quantiles=[0.025, 0.1, 0.25, 0.5, 0.75, 0.9, 0.975]):
    forecast_date = pd.Timestamp(forecast_date)

    long_format_results = []

    # Generate forecasts for the next 4 weeks
    for horizon in range(1, 5):
        target_date = forecast_date + pd.Timedelta(weeks=horizon - 1)

        # Filter data for the target week and neighboring weeks
        weeks = [(target_date + pd.Timedelta(weeks=w)).week for w in range(-window, window + 1)]
        week_data = df[df["week"].isin(weeks)]["value"]

        # Fit Negative Binomial distribution
        r, p = fit_negative_binomial(week_data)
        if r is not None and p is not None:
            # Compute quantiles from the Negative Binomial distribution
            dist = nbinom(r, p)
            quantile_values = dist.ppf(quantiles)
        else:
            # If parameters are invalid, return NaNs for quantiles
            quantile_values = [np.nan] * len(quantiles)

        # Transform week back to a date
        target_end_date = pd.Timestamp.fromisocalendar(target_date.year, target_date.week, 7)  # Sunday

        # Add rows in long format
        for quantile, value in zip(quantiles, quantile_values):
            long_format_results.append(
                {
                    "location": "DE",
                    "age_group": df.age_group.unique()[0],
                    "forecast_date": forecast_date,
                    "target_end_date": target_end_date,
                    "horizon": horizon,
                    "type": "quantile",
                    "quantile": quantile,
                    "value": value,
                }
            )

    # Convert to DataFrame
    return pd.DataFrame(long_format_results)

In [4]:
def compute_historical_average(forecast_date):
    age_groups = ["DE", "00-04", "05-14", "15-34", "35-59", "60-79", "80+"]

    targets, _ = load_realtime_training_data(forecast_date)

    forecasts = pd.DataFrame()
    for age in age_groups:
        col = f"icosari-sari-{age}"
        df_temp = targets[col].to_dataframe().reset_index()
        df_temp["week"] = df_temp["date"].dt.isocalendar().week
        df_temp["age_group"] = "00+" if age == "DE" else age
        df_temp = df_temp.rename(columns={col: "value"})
        forecast = forecast_historical_average(df_temp, forecast_date, window=1)
        forecasts = pd.concat([forecasts, forecast], ignore_index=True)
    return forecasts

In [5]:
out_dir = ROOT / "forecasts" / "historical"
out_dir.mkdir(parents=True, exist_ok=True)

In [6]:
for forecast_date in FORECAST_DATES:
    print(forecast_date)
    forecast = compute_historical_average(forecast_date)
    forecast.to_csv(out_dir / f"{forecast_date}-icosari-sari-historical.csv", index=False)

2023-11-16
2023-11-23
2023-11-30
2023-12-07
2023-12-14
2023-12-21
2024-01-11
2024-01-18
2024-01-25
2024-02-01
2024-02-08
2024-02-15
2024-02-22
2024-02-29
2024-03-07
2024-03-14
2024-03-21
2024-03-28
2024-04-04
2024-04-11
2024-04-18
2024-04-25
2024-05-02
2024-05-09
2024-05-16
2024-05-23
2024-05-30
2024-06-06
2024-06-13
2024-06-20
2024-06-27
2024-07-04
2024-07-11
2024-07-18
2024-07-25
2024-08-01
2024-08-08
2024-08-15
2024-08-22
2024-08-29
2024-09-05
2024-09-12
